In [1]:
import pandas as pd
import cv2
import numpy as np

In [2]:
folder='/udacity_images/'

In [3]:
df=pd.read_csv(folder+'driving_log.csv',\
            names=['center','left','right','steering','trottle','brake','speed'])

In [4]:
correction=0.2
images=np.empty((len(df)*6,160,320,3), dtype=np.uint8)
target=np.empty((len(df))*6, dtype=np.float)

#center
for i,fname in enumerate(df['center'].values):
    images[i]=cv2.imread(fname.replace('/Users/p.tarasov/Desktop/udacity_images/',folder))

#left
for i,fname in enumerate(df['left'].values):
    images[i+len(df)]=cv2.imread(fname.replace('/Users/p.tarasov/Desktop/udacity_images/',folder))
 
#right
for i,fname in enumerate(df['right'].values):
    images[i+2*len(df)]=cv2.imread(fname.replace('/Users/p.tarasov/Desktop/udacity_images/',folder))
 
target[0:len(df)]=df.steering.values
target[len(df):2*len(df)]=df.steering.values+correction
target[2*len(df):3*len(df)]=df.steering.values-correction

target[3*len(df):6*len(df)]=-target[0:3*len(df)]

for i in range(len(df)*3):
    images[i+3*len(df)]=np.fliplr(images[i])


In [ ]:
df2=pd.read_csv('/notebooks/data/driving_log.csv',\
            names=['center','left','right','steering','trottle','brake','speed'])

In [5]:
from keras.models import Sequential
from keras.layers import Flatten,Dense, Convolution2D, Activation, MaxPooling2D, Lambda,Conv2D,Dropout, Cropping2D

Using TensorFlow backend.


In [7]:
model=Sequential()

model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
model.add(Lambda(lambda x: (x / 255.0) - 0.5))


model.add(Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                  strides =(2,2),\
                 padding='Same'))

model.add(Dropout(0.5))

model.add(Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                  strides =(2,2),\
                 padding='Same'))
model.add(Dropout(0.5))


model.add(Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                  strides =(2,2),\
                 padding='Same'))
model.add(Dropout(0.5))

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                  strides =1,\
                 padding='Same'))
model.add(Dropout(0.5))

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                  strides =1,\
                 padding='Same'))
model.add(Dropout(0.5))

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                  strides =1,\
                 padding='Same'))
model.add(Dropout(0.5))

model.add(Flatten())


model.add(Dense(1164, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(100, activation='relu'))

model.add(Dense(50, activation='relu'))

model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1))

In [8]:
model.compile(loss='mse',optimizer='adam')

In [ ]:
model.fit(images, target, validation_split=0.2, shuffle=True, epochs=10,batch_size=128)

Train on 35467 samples, validate on 8867 samples
Epoch 1/10
35467/35467 [==============================] - 45s - loss: 0.0371 - val_loss: 0.0561

In [ ]:
model.save('model_nvidia.h5')

In [32]:
%reload_ext keras.models.Sequential

ImportError: No module named 'keras.models.Sequential'; 'keras.models' is not a package